# Homework 3

We first import the needed libraries and set the elements we need for the rest of the Homework.

In [ ]:
# Used to look for the saved data sheets
DATA_FOLDER = 'Data/' #still need to create this

In [ ]:
# Importing libraries
import os
import pandas as pd
import json
import seaborn as sns #might not be useful
import folium

In [ ]:
folium.__version__ == '0.5.0' # just checking if we have all the right one installed

## Unemployment in Europe

The very first thing we had to think about is what data we wanted to use. Eurostat had a lot of types of information, even more so, had information per country, but also per province/state of some countries. Thus, we had to determine what kind of information - and how much - we wanted to show.

Thus, we decided to take the **yearly** average for the unemployment rates and not the monthly ones, thus getting data from 2016 and not from 2017. For the monthly rates, we could have had for the beginning of this year, but we choose against it.

Secondly, seeing as the idea was to compare the unemployment rate of Europe to that of Switzerland, it would be more appropriate to take the data by country rather than that per province, even more so seeing as Switzerland is cut in three in that dataset. So, to be able to compare it to Switzerland as a whole, we should have Switzerland as only one entity.

## Unemployment in Switzerland

Here, we need several things to be contained in our data, as this is a question that contains several points. First, we need to see each canton individually, but also to have some statistics of people who alreay have a job but search for another one. 

First, on amstat, we will go to the category "Chômeurs" and then look at the details. Then, we will choose "Taux de chômage". We will also choose to look at the whole year in review, so all of 2017, so that we can maybe do a few comapraisons. Thus we choose "À partir du janvier de l'année sous revue". 

We will also choose three indicators: "Chômeurs inscrits", "Chômeurs jeunes" and "Chômeurs de longue durée" to see all the types of unemployed people. We also take both "Demandeur d'emploi" and "Demandeur d'emploi non chômeur" as to know who is or isn't really unemployed.

Next, for the geography part, we select all three (Language, region and canton), mainly so that we can maybe use this dataset later for the Röstigraen question.

Lastly, we also select the option "Classes d'âge 15-24, 25-49, 50 ans et plus" as we will be able to use it to see the different categories of unemplyed people and use it in the last part of the question.

### Differences in unemployment between foreigners and natives

Here again, we have two different questions: one about the difference in employment regarding nationality, and the second regarding age. Seeing as we already have the information regarding age in the first dataset, we do not need to find another one. Regarding nationality, we proceed similarly as above.

Thus, we choose exactly the same information as before but at the last option, we choose "Nationalité" instead of the age classes.

### The Röstigraben